In [39]:
from pyspark import SparkConf, SparkContext 
import pyspark
import sys
from collections import defaultdict


In [41]:
def getData(sc, filename): 
    """
    Load data from raw text file into RDD and transform.
    Hint: transfromation you will use: map(<lambda function>).
    Args:
        sc (SparkContext): spark context.
        filename (string): hw2.txt cloud storage URI.
    Returns:
        RDD: RDD list of tuple of (<User>, [friend1, friend2, ... ]),
        each user and a list of user's friends
    """
    # read text file into RDD
    data = sc.textFile(filename)
    data = data.map(lambda line: line.split(",")).map(lambda x: (x[1],x[2])).map(lambda y: (y[0], y[1].split(' ')))
    data = data.map(lambda x: (x[1][0], x[0]))
    data = data.zipWithIndex().filter(lambda row_index: row_index[1] > 0).keys()
    data = data.groupByKey()
    #.map(lambda x,y: (x[1],[y]))
    #dataset = data.map(lambda line: (line[0], line[1].split(',') if len (line[1]) else []))
    # TODO: implement your logic here
    return data
sc = SparkContext.getOrCreate()
filename = 'gs://final_project123/try.csv'
data_new = getData(sc, filename)


In [42]:
def mapFriends(line):
    """
    List out every pair of mutual friends, also record direct friends.
    Hint:
    For each <User>, record direct friends into a list:
    [(<User>, (friend1, 0)),(<User>, (friend2, 0)), ...],
    where 0 means <User> and friend are already direct friend,
    so you don't need to recommand each other.

    For friends in the list, each of them has a friend <User> in common,
    so for each of them, record mutual friend in both direction:
    (friend1, (friend2, 1)), (friend2, (friend1, 1)),
    where 1 means friend1 and friend2 has a mutual friend <User> in this "line"

    There are possibly multiple output in each input line,
    we applied flatMap to flatten them when using this function.
    Args:
        line (tuple): tuple in data RDD
    Yields:
        RDD: rdd like a list of (A, (B, 0)) or (A, (C, 1))
    """
    friend = line[1]
    user = line[0]
    if friend == []:
        return
    try:
        for i in range(len(friend)):
        # Direct friend
        # TODO: implement your logic here
            yield (int(user),(int(friend[i]),0))
        
            for j in range(i+1, len(friend)):
            # Mutual friend in both direction
            # TODO: implement your logic here
                yield (int(friend[i]),(int(friend[j]),1))
                yield (int(friend[j]),(int(friend[i]),1))
            
            
    except:
        pass
    

In [43]:
data_new.take(5)

[(u'swish41', <pyspark.resultiterable.ResultIterable at 0x7f1dce597890>),
 (u'God_Son80', <pyspark.resultiterable.ResultIterable at 0x7f1dce597390>),
 (u'obj', <pyspark.resultiterable.ResultIterable at 0x7f1dce597a90>),
 (u'ESPNNBApic', <pyspark.resultiterable.ResultIterable at 0x7f1dce597a50>),
 (u'NFL', <pyspark.resultiterable.ResultIterable at 0x7f1dce597f10>)]

In [44]:
getFriends = data_new.mapValues(list)
getFriends.take(5)

[(u'swish41', [u'Ballislife', u'ESPNNBA']),
 (u'God_Son80', [u'ComplexSports', u'darrenrovell']),
 (u'obj', [u'ComplexSports', u'darrenrovell']),
 (u'ESPNNBApic', [u'Ballislife']),
 (u'NFL', [u'dailyrepubIican'])]

In [45]:
vertices=getFriends.map(lambda x:(x[0],))

In [50]:
vertices.collect()

[(u'swish41',),
 (u'God_Son80',),
 (u'obj',),
 (u'ESPNNBApic',),
 (u'NFL',),
 (u'Nike',)]

In [51]:
def f(x): return x

In [52]:
edges = data_new.flatMapValues(f)


In [53]:
edges.collect()

[(u'swish41', u'Ballislife'),
 (u'swish41', u'ESPNNBA'),
 (u'God_Son80', u'ComplexSports'),
 (u'God_Son80', u'darrenrovell'),
 (u'obj', u'ComplexSports'),
 (u'obj', u'darrenrovell'),
 (u'ESPNNBApic', u'Ballislife'),
 (u'NFL', u'dailyrepubIican'),
 (u'Nike', u'ComplexSports')]

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark dataframe") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
v = spark.createDataFrame(vertices, ["id"])

In [ ]:
e = spark.createDataFrame(edges,["src","dst"])